<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/NB15_ML_SL_GLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos Lineares Generalizados (GLM)

## Introdução ao GLM
* O que é um Modelo Linear Generalizado (GLM)?

    * É uma extensão do modelo de regressão linear que permite lidar com variáveis de resposta que não seguem uma distribuição normal.

* O GLM combina:
    * Previsores lineares: Soma ponderada das variáveis independentes.
    * Função de ligação: Relaciona o previsor linear à variável de resposta.
    * Distribuição da variável de resposta: Como Binomial, Poisson, Gamma, etc.

* Por que usar GLM?

    * Modelar dados com distribuições não normais.
    * Lidar com variáveis de resposta contínuas, discretas ou categóricas.
    * Tornar os modelos lineares mais flexíveis para diferentes problemas.

* Exemplo prático:

    * Modelar a probabilidade de um cliente comprar um produto (binomial).
    * Prever o número de chamadas em um call center por hora (Poisson).
    * Prever o número de sinistros para seguradoras (Poisson).


## Explicação teórica:

* Um Modelo Linear Generalizado (GLM) estende os modelos lineares simples para cobrir variáveis-alvo com distribuições diferentes da normal. Ele é composto por:
    * Um componente linear: 𝜂 = 𝛽_0 + 𝛽_1.𝑋_1 + ⋯ + 𝛽_𝑘.𝑋_𝑘.
    * Uma função de ligação que transforma o componente linear para se adequar à distribuição da variável-alvo.
    * Uma distribuição da família exponencial para modelar a variável-alvo (Binomial, Poisson, Gamma, etc.).

* Aplicação prática em seguros:
    * Modelar o número de sinistros por cliente em uma seguradora (distribuição Poisson).
    * Prever o custo médio de sinistros para apólices de seguro saúde (distribuição Gamma).

* Aplicação prática em bancos:
    * Modelar a probabilidade de inadimplência de um cliente em um empréstimo (distribuição Binomial, regressão logística).

# Componentes de um GLM
* Função de ligação: Relaciona o componente linear com a média da distribuição da variável-alvo.

    * Exemplo: Para uma variável binária (compra ou não compra de um seguro), a função de ligação logit é usada.
* Distribuição da variável de resposta:
    * Binomial: Para eventos com sucesso/falha.
    * Poisson: Para contagens ou taxas.
    * Gamma: Para valores contínuos positivos e assimétricos.

* Previsores lineares: Variáveis explicativas como idade, renda ou histórico de crédito.

* Exemplo prático: Em um banco, a probabilidade de inadimplência pode ser modelada com a regressão logística, onde a função de ligação logit transforma a probabilidade em uma escala linear.

# Tipos Comuns de GLM e Aplicações
## Regressão Logística (Binomial):

* Seguro: Modelar a probabilidade de renovação de uma apólice.
* Banco: Estimar a chance de um cliente atrasar um pagamento.

## Regressão de Poisson:

* Seguro: Modelar o número de sinistros por cliente.
* Banco: Estimar o número de transações mensais de um cliente.

## Regressão Gamma:

* Seguro: Estimar o custo médio de sinistros para seguros de saúde.
* Banco: Prever o valor médio de gastos no cartão de crédito.

## Regressão Gaussiana (Linear):

* Seguro: Prever o valor total de prêmios arrecadados em um período.
* Banco: Estimar o valor total de financiamentos concedidos.

# Exemplo
* O dataset a seguir (fictício e criado usando a library faker) com variáveis relevantes para a cobertura de Casco, incluindo fatores de risco para frequência e severidade de sinistros.

In [ ]:
!pip install faker

In [ ]:
import numpy as np
import pandas as pd

# Configurando o gerador de números aleatórios
np.random.seed(42)

# Gerando o dataset
n_samples = 50000
years = np.random.choice(range(2012, 2024), n_samples)
months = np.random.choice(range(1, 13), n_samples)

# Criando variáveis ajustadas
data = {
    "ano": years,
    "mes": months,
    "idade_condutor": np.random.randint(18, 80, n_samples),
    "experiencia_condutor": np.random.randint(1, 60, n_samples),
    "tipo_veiculo": np.random.choice(["hatch", "sedan", "suv", "pickup", "luxo"], n_samples, p=[0.3, 0.4, 0.2, 0.08, 0.02]),
    "regiao": np.random.choice(["norte", "sul", "leste", "oeste", "centro-oeste"], n_samples),
    "cidade": np.random.choice(["cidade_a", "cidade_b", "cidade_c", "cidade_d", "cidade_e"], n_samples),
    "historico_sinistro": np.random.choice([0, 1], n_samples, p=[0.75, 0.25]),
    "km_rodados_ano": np.random.normal(20000, 6000, n_samples).clip(5000, 60000),
    "cobertura_casco": np.random.normal(70000, 25000, n_samples).clip(20000, 200000),
    "tipo_combustivel": np.random.choice(["gasolina", "diesel", "eletrico", "flex"], n_samples, p=[0.4, 0.3, 0.1, 0.2]),
    "estado_civil": np.random.choice(["solteiro", "casado", "divorciado", "viuvo"], n_samples),
    "genero": np.random.choice(["F", "M"], n_samples, p=[0.4, 0.6]),
    "uso": np.random.choice(["profissional", "pessoal"], n_samples, p=[0.3, 0.7]),
    "idade_veiculo": np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], n_samples, p=[0.15, 0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]),
    "numero_de_condutores": np.random.randint(1, 5, n_samples)
}

# Gerando 'frequência de sinistros'
data["frequencia_sinistro"] = (
    0.04 * data["idade_condutor"] +
    0.05 * data["experiencia_condutor"] +
    0.1 * data["historico_sinistro"] +
    0.07 * (data["ano"] - 2012) +
    0.2 * (data["tipo_veiculo"] == "luxo").astype(int) +
    0.15 * (data["regiao"] == "sul").astype(int) +
    0.18 * (data["cidade"] == "cidade_e").astype(int) +
    0.1 * (data["uso"] == "profissional").astype(int) +
    np.random.poisson(0.3, n_samples)
)

# Gerando 'severidade dos sinistros'
data["severidade_sinistro"] = (
    5000 +
    0.5 * data["idade_condutor"] +
    0.6 * np.log(data["km_rodados_ano"] + 1) +
    0.8 * (data["tipo_veiculo"] == "pickup").astype(int) +
    0.7 * (data["tipo_combustivel"] == "diesel").astype(int) +
    0.6 * (data["cidade"] == "cidade_e").astype(int) +
    0.5 * data["experiencia_condutor"] +
    0.5 * (data["uso"] == "profissional").astype(int) +
    0.4 * (data["regiao"] == "oeste").astype(int) +
    0.4 * data["idade_veiculo"] +
    0.3 * data["numero_de_condutores"] +
    0.7 * ((data["estado_civil"] == "solteiro") & (data["idade_condutor"] < 24)).astype(int) +
    np.random.normal(0, 800, n_samples)  # Maior dispersão
)

# Generating the number of accidents with a Poisson distribution
data["numero_acidentes"] = np.random.poisson(
    lam=(
        0.5
        + 0.03 * data["frequencia_sinistro"]
        + 0.02 * (data["idade_condutor"] < 25).astype(int)
        + 0.05 * (data["uso"] == "profissional").astype(int)
    ),
    size=n_samples,
)

# Criando o DataFrame
df = pd.DataFrame(data)

# Exibindo o DataFrame para o usuário
df.head()

### Detalhes Principais do Conjunto de Dados
* Ano (ano) e Mês (mes):
    * Representam os componentes temporais para tendências e sazonalidade.
* Frequências:
    * Incluem um aumento linear com o tempo (0,005 * (ano - 2012)).
    * Adicionam sazonalidade usando ondas senoidais para padrões mensais.
* Severidade:
    * A gravidade aumenta ao longo do tempo em 1% por ano (1 + 0,01 * (ano - 2012)).

## Step 1: Preparar os dados (Data Preprocessing)

In [ ]:
# Creating dummy variables for categorical predictors
df = pd.get_dummies(df, columns=["genero", "regiao", "cidade", "tipo_veiculo", "uso", "tipo_combustivel", "estado_civil"], drop_first=True)


## Step 2: Ajustar os GLMs
### GLM para frequência de sinistros (Distribuição de Poisson)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

X_freq = df.drop(columns=['frequencia_sinistro'])
y_freq = df['frequencia_sinistro']

# Convert all columns in X_freq and y_freq to numeric, coercing errors to NaN
X_freq = X_freq.apply(pd.to_numeric, errors='coerce')
y_freq = pd.to_numeric(y_freq, errors='coerce')

# If you have any date columns, convert them to numeric representation:
# For example, if 'ano' and 'mes' are date components:
X_freq['ano'] = pd.to_numeric(X_freq['ano'], errors='coerce')
X_freq['mes'] = pd.to_numeric(X_freq['mes'], errors='coerce')

# Replace infinite values with NaN
X_freq.replace([np.inf, -np.inf], np.nan, inplace=True)
y_freq.replace([np.inf, -np.inf], np.nan, inplace=True)

# --- The Fix Starts Here ---
# Impute missing values using a more robust strategy
for col in X_freq.select_dtypes(include=np.number).columns:
    # Calculate the mean of non-missing values
    col_mean = X_freq[col].dropna().mean()
    # If the mean is not calculable (all values are NaN), use 0
    if pd.isnull(col_mean):
        col_mean = 0
    X_freq[col] = X_freq[col].fillna(col_mean)

# If y_freq is numeric, impute missing values using its mean
if pd.api.types.is_numeric_dtype(y_freq):
    y_freq = y_freq.fillna(y_freq.mean())
else:
    # If y_freq is not numeric, consider a suitable imputation strategy
    # For example, you might impute with the most frequent value
    y_freq = y_freq.fillna(y_freq.mode()[0])
# --- The Fix Ends Here ---

# Add intercept
X_freq = sm.add_constant(X_freq)

# --- The Fix Starts Here ---
# Check for and handle invalid values in y_freq
# If y_freq contains negative values, shift it to be non-negative for Poisson regression
if y_freq.min() < 0:
    print("Warning: y_freq contains negative values. Shifting to be non-negative.")
    y_freq = y_freq - y_freq.min() + 1e-8  # Add a small positive value to avoid zeros
# --- The Fix Ends Here ---

# Explicitly convert all columns in X_freq to numeric dtypes, handling errors
for col in X_freq.select_dtypes(include=['object']).columns:
    try:
        X_freq[col] = pd.to_numeric(X_freq[col])
    except ValueError:
        print(f"Warning: Could not convert column '{col}' to numeric. Trying to convert to category and then to numeric.")
        try:
            # Attempt to convert to categorical and then to numeric codes
            X_freq[col] = X_freq[col].astype('category').cat.codes
        except ValueError:
            print(f"Warning: Could not convert column '{col}' to numeric even after category conversion. Dropping this column.")
            X_freq = X_freq.drop(columns=[col])

# Ensure y_freq is also numeric
y_freq = pd.to_numeric(y_freq, errors='coerce')

X_freq = pd.DataFrame(X_freq.to_numpy(), columns=X_freq.columns, dtype=float)

# Fit the Poisson GLM
model_freq = sm.GLM(y_freq, X_freq, family=sm.families.Poisson())
result_freq = model_freq.fit()

# Display results
print(result_freq.summary())

# Interpretação do modelo de Poisson para Sinistros

A formulação de um modelo de Poisson para sinistros é geralmente representada da seguinte forma:

$$
\ln(\mu_i) = \beta_0 + \beta_1X_1 + \beta_2X_2 + \dots + \beta_pX_p
$$

Onde:
- $\mu_i$: Frequência esperada de sinistros para a apólice \(i\) (número de sinistros esperados por unidade de exposição, como anos-segurados).
- $X_1, X_2, \dots, X_p$: Variáveis independentes (fatores de risco).
- $\beta_0$: Intercepto do modelo.
- $\beta_1, \beta_2, \dots, \beta_p$: Coeficientes associados às variáveis independentes.
- A função de link logarítmica transforma $\mu_i$ para garantir que as predições sejam sempre positivas (pois o número esperado de sinistros não pode ser negativo).

---

## **1. Compreensão Geral do Modelo**

- **Variável Dependente:** `frequencia_sinistro` (número esperado de sinistros por unidade de exposição).
- **Família do Modelo:** **Poisson**, apropriado para modelar contagens.
- **Link Function:** Logaritmo natural $(\ln)$, garantindo que as predições sejam sempre positivas.
- **Número de Observações:** 50.000.
- **Log-Likelihood:** -83.955, uma métrica útil para comparação com outros modelos.
- **Pseudo R-squared (CS):** 0.2630, indicando que cerca de 26,3% da variação explicada pode ser atribuída ao modelo (um valor relativamente baixo para indicar bom ajuste).
- **Número de Iterações:** 4, sugerindo que o modelo convergiu rapidamente.

---

## **2. Avaliação do Modelo**

- **Deviance:** 4.041,6 — mede o ajuste entre os valores observados e preditos. Menores valores indicam melhor ajuste.
- **Pearson Chi-Square:** 4.260 — avalia a discrepância entre valores observados e preditos. O valor próximo à Deviance sugere consistência no ajuste.

---

## **3. Interpretação dos Coeficientes**

Os coeficientes no modelo de Poisson são apresentados na escala logarítmica. Para interpretar os impactos diretamente na frequência esperada de sinistros, convertemos os coeficientes usando $e^{\text{coef}}$.

---

### **3.1 Intercepto (const)**

- **Coeficiente:** $-32.2560$.
- **Interpretação:** Representa o logaritmo da frequência esperada de sinistros na categoria de referência para todas as variáveis. A frequência base esperada é:

$$
e^{-32.2560} \approx 8.6 \times 10^{-15}
$$

Esse valor muito pequeno sugere que, na ausência de fatores de risco, a frequência é insignificante, o que é esperado em modelos de sinistros baseados em exposição.

---

### **3.2 Variáveis Contínuas**

1. **Ano**:
   - **Coeficiente:** $0.0163$ (**significativo, $p < 0.000$**).
   - **Interpretação:** A cada ano adicional, a frequência esperada de sinistros aumenta em cerca de $e^{0.0163} \approx 1.016$, ou 1,6%.

2. **Idade do Condutor**:
   - **Coeficiente:** $0.0094$ (**significativo, $p < 0.000$**).
   - **Interpretação:** Cada ano adicional na idade do condutor aumenta a frequência esperada em $e^{0.0094} \approx 1.009$, ou 0,9%.

3. **Experiência do Condutor**:
   - **Coeficiente:** $0.0119$ (**significativo, $p < 0.000$**).
   - **Interpretação:** Cada ano adicional de experiência aumenta a frequência esperada em $e^{0.0119} \approx 1.012$, ou 1,2%.

4. **Histórico de Sinistro**:
   - **Coeficiente:** $0.0211$ (**significativo, $p < 0.000$**).
   - **Interpretação:** Cada sinistro adicional no histórico do condutor aumenta a frequência esperada de novos sinistros em $e^{0.0211} \approx 1.021$, ou 2,1%.

---

### **3.3 Variáveis Categóricas**

#### Regiões
- **Região Sul:**
  - **Coeficiente:** $0.0299$ (**significativo, $p < 0.000$**).
  - **Interpretação:** Apólices na Região Sul apresentam uma frequência esperada de sinistros $e^{0.0299} \approx 1.030$, ou 3% maior que a categoria de referência.

- **Demais Regiões:** Nenhuma outra variável regional foi significativa ($p > 0.05$).

#### Cidade
- **Cidade E:**
  - **Coeficiente:** $0.0384$ (**significativo, $p < 0.000$**).
  - **Interpretação:** Apólices nesta cidade têm uma frequência esperada de sinistros $e^{0.0384} \approx 1.039$, ou 3,9% maior que a cidade de referência.

#### Tipo de Veículo
- **Luxo:**
  - **Coeficiente:** $0.0463$ (**significativo, $p = 0.003$**).
  - **Interpretação:** Veículos de luxo têm uma frequência esperada de sinistros $e^{0.0463} \approx 1.047$, ou 4,7% maior que veículos na categoria de referência.

---

### **3.4 Outras Variáveis Significativas**

1. **Uso Profissional**:
   - **Coeficiente:** $0.0241$ (**significativo, $p < 0.000$**).
   - **Interpretação:** Apólices com veículos de uso profissional apresentam uma frequência esperada $e^{0.0241} \approx 1.024$, ou 2,4% maior.

---

### **3.5 Variáveis Não Significativas**

- **Km Rodados por Ano:** $p = 0.698$
- **Cobertura de Casco:** $p = 0.855$
- **Severidade do Sinistro:** $p = 0.931$
- **Combustível e Estado Civil:** Nenhuma categoria apresentou significância estatística ($p > 0.05$).

**Interpretação Geral:**
Essas variáveis não tiveram impacto estatisticamente significativo na frequência de sinistros. Sugere-se revisá-las quanto à qualidade dos dados ou avaliar possíveis interações com outras variáveis.

---

## **4. Insights Chave**

1. **Fatores de Risco Importantes:**
   - **Ano, idade do condutor, experiência do condutor e histórico de sinistros** têm impactos positivos e significativos na frequência de sinistros.
   - **Região Sul, Cidade E, tipo de veículo (luxo) e uso profissional** também aumentam a frequência esperada de sinistros.

2. **Fatores Sem Impacto:**
   - Variáveis como **km rodados**, **cobertura casco**, **severidade do sinistro**, e **combustível** não foram significativas.

3. **Recomendações para Melhorias:**
   - **Transformação de Variáveis:** Variáveis contínuas, como km rodados, podem se beneficiar de transformações (e.g., logarítmica).
   - **Interações:** Testar interações entre idade e experiência do condutor, tipo de veículo e uso profissional, idade e sexo, etc.
   - **Revisar Dados:** Avaliar a qualidade e representatividade dos dados para variáveis não significativas.

---

## **5. Conclusão**

O modelo ajustado apresenta bons insights sobre os fatores que influenciam a frequência de sinistros. Embora o ajuste global (Pseudo R-squared de 26,3%) seja moderado, as variáveis significativas capturam relações importantes que podem ser usadas para ajustar prêmios de seguro. Contudo, a inclusão de interações e modelos alternativos (e.g., Random Forest, XGBoost) pode trazer mais valor ao processo.

### GLM para Severidade (Distribuição Gamma)

In [ ]:
# Define predictors and target for severity
X_sev = X_freq  # Same predictors
y_sev = df["severidade_sinistro"]

# Fit the Gamma GLM
model_sev = sm.GLM(y_sev, X_sev, family=sm.families.Gamma(link=sm.families.links.log()))
result_sev = model_sev.fit()

# Display results
print(result_sev.summary())


### **1. Compreensão Geral do Modelo**
- **Variável Dependente:** `severidade_sinistro` (valor esperado do custo de um sinistro para uma apólice).
- **Família do Modelo:** **Gamma** (escolhida para modelar a severidade devido à natureza contínua e assimétrica dos custos).
- **Link Function:** Logaritmo natural ($\ln$), garantindo que as predições sejam sempre positivas.
- **Número de Observações:** 50.000.
- **Log-Likelihood:** -298,710, usado como métrica de ajuste do modelo.
- **Pseudo R-squared (CS):** 1.000, indicando que o modelo captura quase toda a variação explicada nos dados (valores altos podem indicar ajuste "perfeito", mas com risco de overfitting).

---

### **2. Avaliação das Métricas do Modelo**
- **Deviance:** 18.180 — mede o ajuste do modelo. Valores baixos indicam bom ajuste, mas devem ser comparados entre modelos alternativos.
- **Pearson Chi-Square:** 17.4 — similar à deviance, avalia a discrepância entre valores observados e preditos.
- **Número de Iterações (IRLS):** 10 — o modelo convergiu em um número razoável de iterações, sugerindo boa estabilidade.

---

### **3. Interpretação dos Coeficientes**
Os coeficientes são interpretados no logaritmo natural devido ao link log. Para entender o impacto das variáveis, convertemos os coeficientes usando $e^{\text{coef}}$.

#### **Intercepto (const):**
- **Valor:** $7.4597$.
- **Interpretação:** Representa o logaritmo do valor base da severidade para todas as variáveis categóricas na categoria de referência e variáveis contínuas iguais a zero.
- **Severidade Base:** $e^{7.4597} \approx 1,740$. A severidade média esperada para apólices na categoria de referência é de aproximadamente R$1.740.

---

#### **Variáveis Contínuas**
1. **Idade do Condutor:**
   - **Coeficiente:** $9.053e-06$ (não significativo com $p = 0.053$).
   - **Interpretação:** Pequeno impacto positivo na severidade. Cada ano adicional de idade aumenta a severidade esperada em cerca de $e^{9.053e-06} \approx 1.000009$, ou seja, quase insignificante.

2. **Histórico de Sinistro:**
   - **Coeficiente:** $-0.0001$ (não significativo com $p = 0.484$).
   - **Interpretação:** Não é possível afirmar que o histórico de sinistros reduz a severidade com base neste modelo.

3. **Número de Acidentes:**
   - **Coeficiente:** $-0.0002$ (**significativo com $p = 0.019$**).
   - **Interpretação:** Reduz levemente a severidade esperada. Cada acidente adicional está associado a uma redução de cerca de 0.01% na severidade ($e^{-0.0002} \approx 0.9998$).

---

#### **Variáveis Categóricas**
1. **Região:**
   - Nenhuma das variáveis relacionadas à região (leste, norte, oeste, sul) mostrou significância estatística ($p >0.05$).
   - **Conclusão:** A localização geográfica parece não afetar a severidade de sinistros neste modelo.

2. **Cidade:**
   - Nenhuma das variáveis categóricas relacionadas às cidades foi significativa ($p > 0.05$).

3. **Tipo de Veículo:**
   - **Tipo Luxo:** $-5.545e-05$ ($p =0.928$, não significativo).
   - **Tipo Pickup:** $0.0004$ ($p =0.179$, não significativo).
   - **Tipo Sedan:** $-1.138e-05$ ($p =0.955$, não significativo).
   - **Tipo SUV:** $0.0002$ ($p =0.401$, não significativo).
   - **Conclusão:** O tipo de veículo não apresenta evidências de impacto significativo na severidade.

4. **Tipo de Combustível:**
   - **Flex:** $0.0002$ ($p =0.427$, não significativo).
   - **Gasolina:** $0.0003$ ($p =0.095$, quase significativo).
   - **Conclusão:** Combustíveis parecem ter impacto limitado na severidade de sinistros.

5. **Estado Civil:**
   - Nenhuma variável relacionada ao estado civil foi significativa ($p >0.05$).

---

### **4. Insights Chave**
1. **Variáveis Significativas:**
   - Apenas o número de acidentes teve significância estatística ($p =0.019$).
   - O intercepto sugere uma severidade média esperada alta ($e^{7.4597} \approx R\$1.740$).

2. **Variáveis Não Significativas:**
   - A maioria das variáveis (idade, histórico de sinistros, tipo de veículo, tipo de combustível, etc.) não apresenta significância estatística. Isso pode indicar:
     - Dados insuficientes para identificar padrões claros.
     - Possível multicolinearidade entre os preditores.
     - Necessidade de transformação de variáveis ou interação.

3. **Ajuste do Modelo:**
   - O modelo apresenta bom ajuste geral com base na Deviance e no Log-Likelihood, mas a significância limitada dos preditores sugere a necessidade de revisão.

---

### **5. Recomendações**
1. **Explorar Transformações e Interações:**
   - Verificar se interações (e.g., idade x tipo de veículo) podem melhorar a explicação da severidade.
   - Testar transformações logarítmicas ou categorização de variáveis contínuas.

2. **Revisar Dados:**
   - Investigar se existem problemas de qualidade ou representatividade nos dados (e.g., valores extremos, multicolinearidade).

3. **Comparar com Outros Modelos:**
   - Ajustar modelos alternativos, como Random Forest ou XGBoost, para verificar se padrões não lineares melhor explicam a severidade.

4. **Avaliar Relevância Negocial:**
   - Embora muitas variáveis sejam estatisticamente não significativas, podem ser relevantes do ponto de vista de negócio e devem ser consideradas no prêmio.

---

# Exercícios

1. Identificação de Componentes
Pergunta: Explique os três componentes principais de um Modelo Linear Generalizado (GLM):
- Função de ligação.
- Distribuição da variável de resposta.
- Componente linear.

Objetivo: Revisar os fundamentos teóricos do GLM.

---

2. Escolha de Modelos
Pergunta: Qual modelo GLM seria apropriado para cada cenário abaixo? Justifique sua escolha.
1. Modelar o número de sinistros por cliente.
2. Prever o custo médio de sinistros em seguros de saúde.
3. Estimar a probabilidade de inadimplência em um banco.

Objetivo: Diferenciar entre regressões Poisson, Gamma e Logística.

---

3. Pré-processamento de Dados
Pergunta: Com base no DataFrame abaixo, quais variáveis devem ser transformadas em variáveis dummy antes de ajustar um GLM? Explique por quê.

```python
df.head()
```

| idade | renda_anual | anos_experiencia | possui_casa | tipo_veiculo |
|-------|-------------|------------------|-------------|--------------|
| 34    | 75000       | 12               | 1           | sedan        |
| 45    | 62000       | 20               | 0           | suv          |

Objetivo: Introduzir boas práticas de pré-processamento de dados.

---

4. Ajuste de Regressão Logística
Pergunta: Ajuste um modelo de regressão logística para prever a probabilidade de um cliente atrasar um pagamento (`inadimplencia`) com base em:
- Idade.
- Renda anual.
- Anos de experiência.

Calcule a acurácia do modelo no conjunto de teste.

Objetivo: Implementar e avaliar um GLM binomial.

---

5. Regressão de Poisson
Pergunta: Ajuste um modelo de regressão de Poisson para prever o número de sinistros (`sinistros`) com base nas variáveis:
- Idade.
- Renda anual.
- Possui casa.

Visualize os valores reais versus preditos e interprete os resultados.

Objetivo: Praticar o ajuste e visualização de um GLM Poisson.

---

6. Regressão Gamma
Pergunta: Use a regressão Gamma para modelar o prêmio de seguro (`premio_seguro`) com as seguintes variáveis:
- Anos de experiência.
- Tipo de veículo (use variáveis dummy).
- Número de dependentes.

Avalie o modelo com o Mean Absolute Error (MAE).

Objetivo: Aplicar regressão Gamma para dados contínuos positivos.

---

7. Interpretação de Coeficientes
Pergunta: Considere o modelo ajustado abaixo e explique o que significa um coeficiente de \(0.05\) para `anos_experiencia` na regressão de Poisson.

$
\text{ln}(E(y)) = 1.2 + 0.05 \times \text{anos_experiencia}
$

Objetivo: Interpretar coeficientes no contexto de uma regressão GLM.

---

8. Escolha da Função de Ligação
Pergunta: Quais critérios você utilizaria para selecionar a função de ligação adequada em um GLM? Explique com exemplos práticos.

Objetivo: Entender a importância e o impacto da função de ligação.

---

9. Comparação entre Modelos
Pergunta: Compare os seguintes modelos ajustados aos mesmos dados:
- Regressão Linear.
- Regressão Poisson.
- Regressão Gamma.

Use métricas como Log-Likelihood e AIC. Qual modelo parece mais apropriado? Por quê?

Objetivo: Avaliar a performance relativa de diferentes GLMs.

---

10. Limitações dos GLMs
Pergunta: Quais são as limitações dos Modelos Lineares Generalizados em relação a:
1. Interpretabilidade.
2. Ajuste em grandes volumes de dados.
3. Interações entre variáveis?

Objetivo: Explorar os desafios práticos de usar GLMs.

---